In [20]:
from pathlib import Path
import numpy as np
import pandas as pd
from songs_analyzer.config import get_root_path

In [21]:
songs_data_path = get_root_path() / "data/raw/song_feature_data.csv"

songs_df = pd.read_csv(songs_data_path)

In [23]:
songs_df.head()

,uri,name,artist,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,spotify:track:7fgC7BPn2SzVno2d7ooLGI,Rufus Wainwright Spotify Pride Intro,Various Artists,0,0.000,0.6290,10,-8.432,0,0.0000,0.348,0.000000,0.3830,0.000,0.000,13750,0
1,spotify:track:3l6EkMrhwXPSVEuTDksWB8,Cigarettes And Chocolate Milk,Rufus Wainwright,44,0.328,0.3570,4,-9.939,1,0.0274,0.791,0.000000,0.0832,0.161,97.816,280360,4
2,spotify:track:2PwYIEV1H34mbQBvbnnmLx,Over the Rainbow,Judy Garland,0,0.231,0.0426,8,-21.141,1,0.0426,0.901,0.000136,0.1530,0.199,77.740,166467,4
3,spotify:track:6bFrlORduDtYQ9BlPenV3o,My Baby Just Cares For Me,Nina Simone,0,0.760,0.1960,11,-17.872,0,0.1110,0.812,0.000002,0.0845,0.418,118.348,216689,4
4,spotify:track:3uxg8Hl1NFVySLDL0euMbu,Cre Spoda,Klaus Nomi,16,0.234,0.8580,8,-6.145,0,0.0710,0.337,0.459000,0.2280,0.200,127.745,183947,4
